In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [2]:
## Load the dataset
churn_data=pd.read_csv("Churn_Modelling.csv")
churn_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
### Drop irrelevant columns
churn_data=churn_data.drop(['RowNumber','CustomerId','Surname'],axis=1)
churn_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Encode categorical variables
label_encoder_gender=LabelEncoder()
churn_data['Gender']=label_encoder_gender.fit_transform(churn_data['Gender'])
churn_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# We cant use label encoding for 'Geography' as it has more than two categories and using label encoding will keep 3 countries like this: France-0,Germany-1,Spain-2 so Spain>germany>farnce which is not true.
# so we sue ohe
from sklearn.preprocessing import OneHotEncoder
onehotencoder_geography=OneHotEncoder(sparse_output=False)
onehotencoder_ge=onehotencoder_geography.fit_transform(churn_data[['Geography']])
geography_encoded_df=pd.DataFrame(onehotencoder_ge,columns=onehotencoder_geography.get_feature_names_out(['Geography']))
geography_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
type(onehotencoder_geography)

sklearn.preprocessing._encoders.OneHotEncoder

In [7]:
## Combine one hot encoder columns with the original data
churn_data=pd.concat([churn_data,geography_encoded_df],axis=1)
churn_data=churn_data.drop(['Geography'],axis=1)
churn_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
## Save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehotencoder_geography.pkl','wb') as file:
    pickle.dump(onehotencoder_geography,file)

##### Exited- dependent feature and others- Independent features

In [9]:
## devide the dataset into indepent and dependent features
X=churn_data.drop('Exited',axis=1)
y=churn_data['Exited']

In [10]:
## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
## Scale these features
churn_scaler=StandardScaler()
X_train=churn_scaler.fit_transform(X_train)
X_test=churn_scaler.transform(X_test)

In [12]:
X_train.shape

(8000, 12)

In [13]:
X_test.shape

(2000, 12)

In [14]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [15]:
with open("churn_scaler.pkl",'wb') as file:
    pickle.dump(churn_scaler,file)

In [16]:
# Build a ANN
churn_model= Sequential([Dense(units=64,activation='relu',input_shape=(X_train.shape[1],))])
churn_model.add(BatchNormalization())
churn_model.add(Dropout(0.3))

# Hl2
churn_model.add(Dense(units=32, activation='relu'))
churn_model.add(BatchNormalization())
churn_model.add(Dropout(0.3))

# output layer
churn_model.add(Dense(units=1, activation='sigmoid'))


2025-09-30 16:24:31.086969: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-09-30 16:24:31.087037: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-09-30 16:24:31.087043: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-09-30 16:24:31.087099: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-30 16:24:31.087116: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [17]:
churn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 batch_normalization_1 (Bat  (None, 32)                128       
 chNormalization)                                                
                                                                 
 dropout_1 (Dropout)         (None, 32)                0

In [18]:
churn_opt=tf.keras.optimizers.Adam(learning_rate=0.01)
churn_loss=tf.keras.losses.BinaryCrossentropy()

In [19]:
# Compile the model
churn_model.compile(optimizer=churn_opt,loss=churn_loss,metrics=['accuracy'])

In [20]:
# Set up ttensorboard
churn_log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboad_callbacks=TensorBoard(churn_log_dir,histogram_freq=1) # for every epoch it will log the data

In [21]:
# Set early stopping
early_stopping=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [22]:
# Training the model
churn_model_history=churn_model.fit(X_train,y_train,validation_split=0.2,epochs=100,callbacks=[early_stopping,tensorboad_callbacks])

Epoch 1/100


2025-09-30 16:24:31.533259: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-09-30 16:24:31.564359: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


200/200 [==============================] - 2s 6ms/step - loss: 0.4916 - accuracy: 0.7808 - val_loss: 0.4535 - val_accuracy: 0.8106
Epoch 2/100
200/200 [==============================] - 2s 6ms/step - loss: 0.4916 - accuracy: 0.7808 - val_loss: 0.4535 - val_accuracy: 0.8106
Epoch 2/100
200/200 [==============================] - 1s 5ms/step - loss: 0.4634 - accuracy: 0.7983 - val_loss: 0.4427 - val_accuracy: 0.8106
Epoch 3/100
200/200 [==============================] - 1s 5ms/step - loss: 0.4634 - accuracy: 0.7983 - val_loss: 0.4427 - val_accuracy: 0.8106
Epoch 3/100
200/200 [==============================] - 1s 5ms/step - loss: 0.4663 - accuracy: 0.7975 - val_loss: 0.4434 - val_accuracy: 0.8163
Epoch 4/100
200/200 [==============================] - 1s 5ms/step - loss: 0.4663 - accuracy: 0.7975 - val_loss: 0.4434 - val_accuracy: 0.8163
Epoch 4/100
200/200 [==============================] - 1s 5ms/step - loss: 0.4615 - accuracy: 0.7952 - val_loss: 0.4276 - val_accuracy: 0.8075
Epoch 5/100

In [23]:
churn_model.save("churn_model.h5")

/Users/dhruvyellanki/Documents/Projects/churn_modelling__ANN_project/churn_venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
# load tensor board extensions
%load_ext tensorboard


In [25]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 80417), started 1:07:14 ago. (Use '!kill 80417' to kill it.)